<a href="https://colab.research.google.com/github/afeef-shaikh/fuzzy-matching-tutorial/blob/main/Tutorial_Cleaning_Messy_Names_with_Dedupe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 👋 Welcome to the Fuzzy Matching Tutorial!

In this tutorial, we’ll walk through how to clean and cluster messy names using the `dedupe` Python library.  
You’ll learn:

- Why duplicate or inconsistent names are a common problem in real-world data.
- How to clean text fields (like company names or organization names).
- How to use `dedupe` to identify and merge similar names automatically.


### **Step 1: Generate a Sample Dataset**

To make this tutorial easy to understand, we will generate a **fake dataset** of 30 fictional company names — including some that are intentionally messy or duplicated (e.g., "Google Inc", "Gogle", "GOOGLE", etc.).

This simulates the real-world problem of matching messy name data from job postings, customer records, or service providers.

In [2]:
# Generate a messy fake dataset

import pandas as pd
import random

# Define some clean company names
base_names = [
    "Google", "Amazon", "Meta", "Tesla", "Apple", "Netflix", "IBM",
    "Microsoft", "Oracle", "Facebook"
]

# Create messy variations
def make_messy(name):
    variants = [
        name.lower(),
        name.upper(),
        name + " Inc.",
        name + " Co",
        name.replace("e", "3"),
        name + " LLC",
        name.replace("o", "0"),
        name + " Corporation",
        name[:3] + " " + name[3:],  # add space
        name + " " + random.choice(["Ltd", "Group", "Holdings"])
    ]
    return random.choice(variants)

# Generate a dataset with 30 rows (including duplicates and messy entries)
random.seed(42)
messy_data = [make_messy(random.choice(base_names)) for _ in range(30)]

# Create DataFrame
df = pd.DataFrame({"OrganizationName": messy_data})

# Save to CSV
df.to_csv("fake_organizations_dataset.csv", index=False)

print("✅ Dataset generated with 30 messy organization names.")


✅ Dataset generated with 30 messy organization names.


In [6]:
# Preview the generated data

from IPython.display import display

display(df)

,OrganizationName
0,Amazon
1,Tesla Inc.
2,Ama zon
3,Amaz0n
4,GOOGLE
5,Tes la
6,Fac ebook
7,Tes la
8,IBM Corporation
9,facebook


### **Step 2: Basic Cleaning with Rule-Based Logic**

Before we do any matching, we need to clean up our organization names.

This includes things like:

* Lowercasing the text
* Removing common suffixes like "Inc.", "LLC", "Co."
* Fixing extra spaces and punctuation

This makes sure we don’t accidentally treat “Google Inc.” and “GOOGLE” as two completely different names.

In [7]:
# Step 2: Clean organization names with rule-based logic

import re

# Define cleaning function
def clean_org_name(name):
    name = name.lower()  # convert to lowercase
    name = re.sub(r"\b(inc|llc|co|corp|corporation|ltd|group|holdings)\b\.?", "", name)  # remove suffixes
    name = re.sub(r"[^\w\s]", "", name)  # remove punctuation
    name = re.sub(r"\s+", " ", name)  # fix multiple spaces
    name = name.strip()  # trim whitespace
    return name

# Apply cleaning
df['Organization_Cleaned'] = df['OrganizationName'].apply(clean_org_name)

# Preview cleaned data
df[['OrganizationName', 'Organization_Cleaned']].head(10)


,OrganizationName,Organization_Cleaned
0,Amazon,amazon
1,Tesla Inc.,tesla
2,Ama zon,ama zon
3,Amaz0n,amaz0n
4,GOOGLE,google
5,Tes la,tes la
6,Fac ebook,fac ebook
7,Tes la,tes la
8,IBM Corporation,ibm
9,facebook,facebook


### **Step 3: Prepare the Data for Dedupe**


The **dedupe** library expects the data in a specific format: a Python dictionary where each record has an ID and its fields.

We’ll convert our cleaned organization names into that format now.

In [21]:
# Step 3: Prepare data for dedupe

# Drop duplicates and reset index before converting to dictionary format
# Resetting index creates a new sequential index (0, 1, 2...)
data_dedupe_df = df[['Organization_Cleaned']].drop_duplicates().reset_index(drop=True)
data_dedupe = data_dedupe_df.to_dict(orient='index')

# Preview sample data for dedupe
print("Sample records to be deduplicated:")
for key in list(data_dedupe.keys()):
    print(key, ":", data_dedupe[key])

Sample records to be deduplicated:
0 : {'Organization_Cleaned': 'amazon'}
1 : {'Organization_Cleaned': 'tesla'}
2 : {'Organization_Cleaned': 'ama zon'}
3 : {'Organization_Cleaned': 'amaz0n'}
4 : {'Organization_Cleaned': 'google'}
5 : {'Organization_Cleaned': 'tes la'}
6 : {'Organization_Cleaned': 'fac ebook'}
7 : {'Organization_Cleaned': 'ibm'}
8 : {'Organization_Cleaned': 'facebook'}
9 : {'Organization_Cleaned': 'meta'}
10 : {'Organization_Cleaned': 'netflix'}
11 : {'Organization_Cleaned': 'apple'}
12 : {'Organization_Cleaned': 'oracle'}
13 : {'Organization_Cleaned': 'microsoft'}
14 : {'Organization_Cleaned': 't3sla'}
15 : {'Organization_Cleaned': 'app le'}


In [17]:
print(data_dedupe)

{0: {'Organization_Cleaned': 'amazon'}, 1: {'Organization_Cleaned': 'tesla'}, 2: {'Organization_Cleaned': 'ama zon'}, 3: {'Organization_Cleaned': 'amaz0n'}, 4: {'Organization_Cleaned': 'google'}, 5: {'Organization_Cleaned': 'tes la'}, 6: {'Organization_Cleaned': 'fac ebook'}, 7: {'Organization_Cleaned': 'ibm'}, 8: {'Organization_Cleaned': 'facebook'}, 9: {'Organization_Cleaned': 'meta'}, 10: {'Organization_Cleaned': 'netflix'}, 11: {'Organization_Cleaned': 'apple'}, 12: {'Organization_Cleaned': 'oracle'}, 13: {'Organization_Cleaned': 'microsoft'}, 14: {'Organization_Cleaned': 't3sla'}, 15: {'Organization_Cleaned': 'app le'}}


### **Step 4: Install and Import dedupe**

The dedupe package helps us find similar text entries using machine learning.
We’ll install it and import the tools we need.

In [9]:
# Step 4: Install and import dedupe

!pip install -q dedupe

import dedupe
from dedupe import Dedupe
from dedupe.variables import String


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.3/347.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.6 MB/s eta

### **Step 5: Define the Fields We Want to Match**

We tell **dedupe** what kind of data we want to compare.
In our case, it’s just one field: the cleaned organization name.
We’ll define that as a *String* type, so it uses text similarity metrics like Levenshtein distance.

In [10]:
# Step 5: Define matching fields
fields = [String('Organization_Cleaned')]

# Initialize deduper
deduper = Dedupe(fields)


### **Step 6: Prepare Training for Matching**

Now we give dedupe a chance to “look” at the data and understand its structure.
This step gets everything ready for labeling.

In [11]:
# Step 6: Prepare training
deduper.prepare_training(data_dedupe)


### **Step 7: Manually Label Match vs Distinct Pairs**

To train **dedupe**, we show it examples of:

* Names that **are the same** (e.g., "gogle" and "google")

* Names that **are different** (e.g., "meta" and "tesla")

This helps it learn the difference between typos vs truly different names.

In [22]:
# Step 7: Label training examples (matches and distincts)

labeled_examples = {
    'match': [
        (data_dedupe[0], data_dedupe[2]),  # amazon vs ama zon
        (data_dedupe[0], data_dedupe[3]),  # amazon vs amaz0n
        (data_dedupe[1], data_dedupe[5]),  # tesla vs tes la
        (data_dedupe[1], data_dedupe[14]), # tesla vs t3sla
        (data_dedupe[8], data_dedupe[6]),  # facebook vs fac ebook
        (data_dedupe[11], data_dedupe[15]) # apple vs app le
    ],
    'distinct': [
        (data_dedupe[0], data_dedupe[1]),  # amazon vs tesla
        (data_dedupe[4], data_dedupe[8]),  # google vs facebook
        (data_dedupe[7], data_dedupe[10]), # ibm vs netflix
        (data_dedupe[12], data_dedupe[13]) # oracle vs microsoft
    ]
}

deduper.mark_pairs(labeled_examples)


### **Step 8: Train the Dedupe Model**

Now we train the model using the labeled examples.

Dedupe will learn which patterns indicate a match (e.g. small typos, spacing changes) vs a non-match.

In [23]:
# Step 8: Train the deduper model
deduper.train()

print("✅ Training complete!")


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


✅ Training complete!


### **Step 9: Set a Similarity Threshold & Cluster Records**

Next, we’ll choose a similarity threshold and use it to group similar names into clusters.

A threshold of **1.0** = only exact matches.

A lower threshold (e.g. **0.7**) allows for more fuzzy matches.

In [31]:
# Step 9: Cluster using a similarity threshold
threshold = 0.7

# Cluster the cleaned names
clusters = deduper.partition(data_dedupe, threshold)

# Print basic stats
print(f"✅ Number of clusters found: {len(clusters)}")


✅ Number of clusters found: 12


### **Step 10: View and Inspect Clusters**

Here we print each cluster, showing:
* The **canonical name** (most common variant)
* The **variations** that were grouped together


In [29]:
# Step 10: Print clusters in a clean format

for idx, (record_ids, scores) in enumerate(clusters, start=1):
    names = [data_dedupe[rid]['Organization_Cleaned'] for rid in record_ids]
    canonical_name = max(set(names), key=names.count)

    print(f"\nCluster {idx}:")
    print(f"Canonical Name: {canonical_name}")
    print("Variations:")
    for name in names:
        if name != canonical_name:
            print(" -", name)



Cluster 1:
Canonical Name: amazon
Variations:
 - amaz0n

Cluster 2:
Canonical Name: tesla
Variations:
 - t3sla

Cluster 3:
Canonical Name: apple
Variations:
 - app le

Cluster 4:
Canonical Name: facebook
Variations:
 - fac ebook

Cluster 5:
Canonical Name: ama zon
Variations:

Cluster 6:
Canonical Name: google
Variations:

Cluster 7:
Canonical Name: tes la
Variations:

Cluster 8:
Canonical Name: ibm
Variations:

Cluster 9:
Canonical Name: meta
Variations:

Cluster 10:
Canonical Name: netflix
Variations:

Cluster 11:
Canonical Name: oracle
Variations:

Cluster 12:
Canonical Name: microsoft
Variations:


### **Step 11: Apply Canonical Names to the Full Dataset**

We now map each original organization name to its cleaned canonical name.

This step is important — it ensures that all duplicates are collapsed into a consistent version for analysis.

In [30]:
# Step 11: Map canonical names to original DataFrame

# Create mapping
canonical_names = {}

for record_ids, _ in clusters:
    cluster_names = [data_dedupe[rid]['Organization_Cleaned'] for rid in record_ids]
    canonical = max(set(cluster_names), key=cluster_names.count)
    for name in cluster_names:
        canonical_names[name] = canonical

# Apply to DataFrame
df['Canonical_Org'] = df['Organization_Cleaned'].map(canonical_names).fillna(df['Organization_Cleaned'])

# View final results
df[['OrganizationName', 'Organization_Cleaned', 'Canonical_Org']].head(10)


,OrganizationName,Organization_Cleaned,Canonical_Org
0,Amazon,amazon,amazon
1,Tesla Inc.,tesla,tesla
2,Ama zon,ama zon,ama zon
3,Amaz0n,amaz0n,amazon
4,GOOGLE,google,google
5,Tes la,tes la,tes la
6,Fac ebook,fac ebook,facebook
7,Tes la,tes la,tes la
8,IBM Corporation,ibm,ibm
9,facebook,facebook,facebook


### **(Optional) Export Final Cleaned Dataset**

If you want to save your cleaned dataset for future use or share it, you can export it as a CSV.



In [32]:
df_clean = df.drop(columns=['Organization_Cleaned']).rename(columns={'Canonical_Org': 'Organization_Clean'})
df_clean.to_csv("cleaned_org_dataset.csv", index=False)

print("✅ Cleaned dataset saved as 'cleaned_org_dataset.csv'")


✅ Cleaned dataset saved as 'cleaned_org_dataset.csv'
